In [2]:
from googlesearch import search

In [3]:
query = "kojima"
results_list = []
for i in search(query,        # The query you want to run
                tld = 'com',  # The top level domain
                lang = 'en',  # The language
                num = 10,     # Number of results per page
                start = 0,    # First result to retrieve
                stop = 10,  # Last result to retrieve
                pause = 2.0,  # Lapse between HTTP requests
               ):
    results_list.append(i)
    print(i)

http://t3.gstatic.com/licensed-image?q=tbn:ANd9GcTHHZI3_Bhpn16_rz9lQJKbpFFPbxbEdHF6w2A_NCMWVoMstdw4vaeTclm_gRg5
https://en.wikipedia.org/wiki/Hideo_Kojima
https://en.wikipedia.org/wiki/Policenauts
https://en.wikipedia.org/wiki/Zone_of_the_Enders
https://en.wikipedia.org/wiki/Penguin_Adventure
http://www.kojimaproductions.jp/en/
https://www.videogameschronicle.com/news/hideo-kojimas-self-made-death-standing-directors-cut-trailer-shows-off-its-new-wide-mode/
https://www.gamepur.com/news/death-stranding-directors-cut-final-trailer-edited-by-hideo-kojima-shows-widescreen-mode
https://www.rockpapershotgun.com/hideo-kojima-is-a-luxury-brand
https://twitter.com/HIDEO_KOJIMA_EN?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor


In [14]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlparse, urljoin

In [5]:

html_page = requests.get(results_list[0])

soup = BeautifulSoup(html_page.text, "html.parser")

urls = []
for link in soup.find_all('a'):
    print(link.get('href'))
print(urls)


[]


In [ ]:
# download robots.txt
# multi threading
# download html for each page


In [17]:
def get_links(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    links = []
    # base = soup.find_all('base')[0].get('href')
    # print(base)
    for link in soup.find_all('a'):
        #filter the links
        #non-url
        #modify relative url to common url

        links.append(link.get('href'))
    for link in links:
        parse_result=urlparse(link)
        print(parse_result)
        if parse_result.scheme=="":
            print(urljoin(url,link))
    return links
get_links('http://google.com')

ParseResult(scheme='http', netloc='www.google.com', path='/imghp', params='', query='hl=en&tab=wi', fragment='')
ParseResult(scheme='http', netloc='maps.google.com', path='/maps', params='', query='hl=en&tab=wl', fragment='')
ParseResult(scheme='https', netloc='play.google.com', path='/', params='', query='hl=en&tab=w8', fragment='')
ParseResult(scheme='http', netloc='www.youtube.com', path='/', params='', query='gl=US&tab=w1', fragment='')
ParseResult(scheme='https', netloc='news.google.com', path='/', params='', query='tab=wn', fragment='')
ParseResult(scheme='https', netloc='mail.google.com', path='/mail/', params='', query='tab=wm', fragment='')
ParseResult(scheme='https', netloc='drive.google.com', path='/', params='', query='tab=wo', fragment='')
ParseResult(scheme='https', netloc='www.google.com', path='/intl/en/about/products', params='', query='tab=wh', fragment='')
ParseResult(scheme='http', netloc='www.google.com', path='/history/optout', params='', query='hl=en', fragment='

['http://www.google.com/imghp?hl=en&tab=wi',
 'http://maps.google.com/maps?hl=en&tab=wl',
 'https://play.google.com/?hl=en&tab=w8',
 'http://www.youtube.com/?gl=US&tab=w1',
 'https://news.google.com/?tab=wn',
 'https://mail.google.com/mail/?tab=wm',
 'https://drive.google.com/?tab=wo',
 'https://www.google.com/intl/en/about/products?tab=wh',
 'http://www.google.com/history/optout?hl=en',
 '/preferences?hl=en',
 'https://accounts.google.com/ServiceLogin?hl=en&passive=true&continue=http://www.google.com/&ec=GAZAAQ',
 '/advanced_search?hl=en&authuser=0',
 '/intl/en/ads/',
 '/services/',
 '/intl/en/about.html',
 '/intl/en/policies/privacy/',
 '/intl/en/policies/terms/']

In [ ]:
#priority queue
#class ScoredUrl
class ScoredUrl:
    def __init__(self, url):
        self.url=url
        self.novelty=10
        self.importance=1

    def crawled(self):
        self.novelty-=1
        self.importance+=1
    
    def referred(self):
        self.importance+=1

    def canCrawl(self):

    def crawl(self):
        return get_links(self.url)

